# Main

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import re
from tqdm import tqdm
import requests
import datetime
import time
from collections import Counter
from random import uniform, randint
import warnings

warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger,
                     NewsNERTagger, Doc)

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import (ConfusionMatrixDisplay, confusion_matrix,
                             f1_score, classification_report)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import tensorflow as tf
import tensorflow_addons as tfa
import keras

In [2]:
username = '...'
password = '...'
host = '...'
port = '...'
database = '...'
conn_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

In [5]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True
companies_df = pd.read_sql_query('SELECT * FROM data.companies_tickets;', conn)
conn.close()

print(companies_df.shape)
companies_df.head()

(7542, 9)


,website,section,url,header,body,tags,body_length,datetime,company
0,РИА,Экономика,https://ria.ru/20230103/banki-1842907991.html,Эксперт рассказал о развитии исламского банкин...,Порядка 10-15 филиалов исламского банкинга м...,"['Экономика', 'Россия', 'Сбербанк России']",1759,2023-01-03 10:16:00,SBER
1,РИА,Экономика,https://ria.ru/20230103/gazprom-1842940413.html,"Экспорт ""Газпрома"" снизился до минимума с конц...","Объём экспорта ""Газпрома"" в прошедшем году с...","['Экономика', 'Газпром', 'Россия']",1299,2023-01-03 14:22:00,GAZP
2,РИА,Экономика,https://ria.ru/20230104/shelf-1843063413.html,Ямальской шельфовой компании предоставили два ...,Правительство РФ предоставило Ямальской шель...,"['Экономика', 'Карское море', 'Россия']",1719,2023-01-04 16:36:00,ROSN
3,Интерфакс,ЭКОНОМИКА,https://www.interfax.ru/business/879947,"""Соллерс"" возобновил после новогодних каникул ...","- Российская автомобилестроительная группа ""...","['Соллерс', 'Соллерс Алабуга', 'Татарстан']",2243,2023-01-09 12:28:00,SVAV
4,Kommersant,Фондовый рынок,https://www.kommersant.ru/doc/5757640,Нестабильный рост,Минувший год запомнится инвесторам как время с...,None,12414,2023-01-06 10:02:00,SBER


# Get data for different deltas

In [6]:
# To send get requests consequently in time
companies_df = companies_df.sort_values(by=['datetime'])

In [17]:
companies_string = 'http://iss.moex.com/iss/engines/stock/markets/shares/securities/{}/candles.json?from={}&till={}&interval=1'
companies_df['price_release'] = np.nan
time_deltas = [5, 10, 15, 30, 45, 60, 75, 90]

for timedelta in tqdm(time_deltas, total=len(time_deltas)):
    companies_df[f'price_lag_{timedelta}'] = np.nan

    for i in range(len(companies_df)):
        start = companies_df['datetime'][i]
        end = companies_df['datetime'][i] + datetime.timedelta(minutes=timedelta)
        company = companies_df['company'][i]
        connect_timeout = 300
        read_timeout = 120
        n = requests.get(companies_string.format(company, start, end), timeout=(connect_timeout, read_timeout)).json()
        try:
            companies_df.loc[i, 'price_release'] = n['candles']['data'][0][0]
            companies_df.loc[i, f'price_lag_{timedelta}'] = n['candles']['data'][-1][0]
        except IndexError:
            continue
    #     if i % randint() == 0:
    #         delay = uniform(5, 10)
    #         time.sleep(delay)

In [35]:
companies_df = companies_df[companies_df['price_release'].notna()]
companies_df.to_csv(f'deltas_test.csv', index=False)

In [66]:
companies_df = pd.read_csv('deltas_test.csv')

# For convinience (in fact, does not make difference)
companies_df = companies_df.drop(['tags'], axis=1).dropna()
companies_df.reset_index(inplace=True)

# Text preprocessing

## Setup

In [70]:
additional_stopwords = [
    'которых', 'которые', 'твой', 'которой', 'которого', 'сих', 'ком', 'свой',
    'твоя', 'этими', 'слишком', 'нами', 'всему', 'будь', 'саму', 'чаще',
    'ваше', 'сами', 'наш', 'затем', 'самих', 'наши', 'ту', 'каждое', 'мочь',
    'весь', 'этим', 'наша', 'своих', 'оба', 'который', 'зато', 'те', 'этих',
    'вся', 'ваш', 'такая', 'теми', 'ею', 'которая', 'нередко', 'каждая',
    'также', 'чему', 'собой', 'самими', 'нем', 'вами', 'ими', 'откуда',
    'такие', 'тому', 'та', 'очень', 'сама', 'нему', 'алло', 'оно', 'этому',
    'кому', 'тобой', 'таки', 'твоё', 'каждые', 'твои', 'нею', 'самим', 'ваши',
    'ваша', 'кем', 'мои', 'однако', 'сразу', 'свое', 'ними', 'всё', 'неё',
    'тех', 'хотя', 'всем', 'тобою', 'тебе', 'одной', 'другие', 'само', 'эта',
    'самой', 'моё', 'своей', 'такое', 'всею', 'будут', 'своего', 'кого',
    'свои', 'мог', 'нам', 'особенно', 'её', 'самому', 'наше', 'кроме',
    'вообще', 'вон', 'мною', 'никто', 'это'
]

stop_words = stopwords.words('russian') + additional_stopwords

In [71]:
# Natasha <3
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

ner_tagger = NewsNERTagger(emb)

In [72]:
del_n = re.compile('\n')
del_tags = re.compile('<[^>]*>')
del_brackets = re.compile('\([^)]*\)')
clean_text = re.compile('[^а-яa-z\s]')
del_spaces = re.compile('\s{2,}')


def prepare_text(text):
    text = del_n.sub(' ', str(text).lower())
    text = del_tags.sub('', text)
    text = del_brackets.sub('', text)
    res_text = clean_text.sub('', text)
    return del_spaces.sub(' ', res_text)


def del_stopwords(text):
    clean_tokens = tuple(
        map(lambda x: x if x not in stop_words else '', word_tokenize(text)))
    res_text = ' '.join(clean_tokens)
    return res_text


def lemmatize(text):
    text = Doc(text)
    text.segment(segmenter)
    text.tag_morph(morph_tagger)
    for token in text.tokens:
        token.lemmatize(morph_vocab)
    text.tag_ner(ner_tagger)
    for span in text.spans:
        span.normalize(morph_vocab)
    return ' '.join([token.lemma for token in text.tokens])

In [73]:
def preprocess_and_save(df, save_name):
    txt_lst = []
    txts = df.body.copy()

    for text in tqdm(txts):
        text = prepare_text(text)
        text = del_stopwords(text)
        text = lemmatize(text)
        txt_lst.append(text)
    
    # Save locally file with preprocessed texts (for convenience)
    df_safe = pd.concat(
        [df, pd.DataFrame(txt_lst, columns=['text_clear'])], axis=1)

    df_safe.to_csv(f'deltas_test_{save_name}.csv', index=False)

In [74]:
def define_classes(df, main_col, class_col, p=0.05):
    df_ret = df.copy()

    q_l = df_ret[main_col].quantile(p)
    q_u = df_ret[main_col].quantile(1 - p)

    df_ret[class_col] = 1
    df_ret.loc[df_ret[main_col] <= q_l, class_col] = 0
    df_ret.loc[df_ret[main_col] >= q_u, class_col] = 2

    print(df_ret[class_col].value_counts())
    
    return df_ret

## Preprocessing

In [75]:
preprocess_and_save(companies_df, 'comp')

100%|██████████████████████████████████████████████| 6934/6934 [09:20<00:00, 12.38it/s]


In [80]:
df_comp = pd.read_csv('deltas_test_comp.csv')

for i, text in enumerate(df_comp.text_clear):
    df_comp.loc[i, 'text_clear'] = prepare_text(text).strip()

# Models testing with different deltas

## Setup

In [82]:
for timedelta in time_deltas:
    df_comp[f'price_diff_{timedelta}'] = df_comp[f'price_lag_{timedelta}'] - df_comp['price_release']
    df_comp[f'price_diff_percent_{timedelta}'] = df_comp[f'price_diff_{timedelta}'] / df_comp['price_release'] * 100
    
    df_comp = define_classes(df_comp, f'price_diff_percent_{timedelta}', f'price_diff_cat_{timedelta}', 0.05)

price_diff_cat_5
1    6240
0     347
2     347
Name: count, dtype: int64
price_diff_cat_10
1    6240
0     347
2     347
Name: count, dtype: int64
price_diff_cat_15
1    6240
2     347
0     347
Name: count, dtype: int64
price_diff_cat_30
1    6240
2     347
0     347
Name: count, dtype: int64
price_diff_cat_45
1    6240
2     347
0     347
Name: count, dtype: int64
price_diff_cat_60
1    6240
0     347
2     347
Name: count, dtype: int64
price_diff_cat_75
1    6240
2     347
0     347
Name: count, dtype: int64
price_diff_cat_90
1    6240
2     347
0     347
Name: count, dtype: int64


In [111]:
def model_my_lstm_fit_predict(y, X, epochs=20):
    texts = X.copy()
    labels = y.copy()

    X_train, X_test, y_train, y_test = train_test_split(texts,
                                                        labels,
                                                        random_state=42,
                                                        test_size=0.25,
                                                        stratify=labels)

    max_words = 10000  # max number of words to use in the vocabulary
    max_len = 500  # max length of each text (in terms of number of words)
    embedding_dim = 500  # dimension of word embeddings
    lstm_units = 32  # number of units in the LSTM layer
    num_classes = len(set(labels))  # number of classes

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)

    sequences_train = tokenizer.texts_to_sequences(X_train)
    sequences_test = tokenizer.texts_to_sequences(X_test)

    X_train = pad_sequences(sequences_train, maxlen=max_len)
    X_test = pad_sequences(sequences_test, maxlen=max_len)

    y_train = pd.get_dummies(y_train).values
    y_test = pd.get_dummies(y_test).values

    keras.backend.clear_session()

    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(LSTM(lstm_units))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(clipvalue=0.7),
                  metrics=[tfa.metrics.F1Score(num_classes=num_classes)])

    scheduler = keras.callbacks.LearningRateScheduler(
        tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=6e-5, decay_rate=0.85, decay_steps=100))

    weights = compute_class_weight(class_weight='balanced',
                                   classes=np.unique(labels),
                                   y=labels)

    weights = {i: weight for i, weight in enumerate(weights)}

    model.fit(X_train,
              y_train,
              batch_size=32,
              epochs=epochs,
              callbacks=[scheduler],
              class_weight=weights,
              validation_data=(X_test, y_test))

    return model

## time_delta==5

In [112]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_5'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 85s 501ms/step - loss: 1.0973 - f1_score: 0.3173 - val_loss: 1.0771 - val_f1_score: 0.3498 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 83s 508ms/step - loss: 1.0821 - f1_score: 0.4648 - val_loss: 1.0544 - val_f1_score: 0.3658 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 90s 551ms/step - loss: 1.0544 - f1_score: 0.5493 - val_loss: 0.9451 - val_f1_score: 0.3983 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 96s 591ms/step - loss: 0.9699 - f1_score: 0.5614 - val_loss: 0.8639 - val_f1_score: 0.3726 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 93s 571ms/step - loss: 0.8570 - f1_score: 0.6241 - val_loss: 0.6824 - val_f1_score: 0.3807 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 93s 571ms/step - loss: 0.7498 - f1_score: 0.6604 - val_loss: 0.6404 - val_f1_score: 0.3824 - lr: 5.9514e-05
Epoch 7/20
163/163 [========

## time_delta==10

In [113]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_10'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 102s 602ms/step - loss: 1.0970 - f1_score: 0.2976 - val_loss: 1.0802 - val_f1_score: 0.3433 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 95s 581ms/step - loss: 1.0813 - f1_score: 0.4280 - val_loss: 1.0629 - val_f1_score: 0.3522 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 97s 593ms/step - loss: 1.0524 - f1_score: 0.5287 - val_loss: 1.0128 - val_f1_score: 0.3645 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 96s 590ms/step - loss: 0.9551 - f1_score: 0.4910 - val_loss: 0.8236 - val_f1_score: 0.3435 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 95s 582ms/step - loss: 0.8019 - f1_score: 0.4709 - val_loss: 0.7856 - val_f1_score: 0.3327 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 95s 583ms/step - loss: 0.7011 - f1_score: 0.5027 - val_loss: 0.8135 - val_f1_score: 0.3459 - lr: 5.9514e-05
Epoch 7/20
163/163 [=======

## time_delta==15

In [114]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_15'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 87s 512ms/step - loss: 1.0981 - f1_score: 0.3023 - val_loss: 1.0785 - val_f1_score: 0.3411 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 75s 461ms/step - loss: 1.0833 - f1_score: 0.4404 - val_loss: 1.0589 - val_f1_score: 0.3900 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 77s 475ms/step - loss: 1.0512 - f1_score: 0.5335 - val_loss: 0.8678 - val_f1_score: 0.3876 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 75s 463ms/step - loss: 0.9707 - f1_score: 0.5400 - val_loss: 0.8777 - val_f1_score: 0.3828 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 84s 514ms/step - loss: 0.8769 - f1_score: 0.5679 - val_loss: 0.6406 - val_f1_score: 0.4051 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 75s 461ms/step - loss: 0.7805 - f1_score: 0.6081 - val_loss: 0.6663 - val_f1_score: 0.3914 - lr: 5.9514e-05
Epoch 7/20
163/163 [========

## time_delta==30

In [115]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_30'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 107s 633ms/step - loss: 1.0974 - f1_score: 0.3065 - val_loss: 1.0786 - val_f1_score: 0.3448 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 102s 625ms/step - loss: 1.0826 - f1_score: 0.4470 - val_loss: 1.0554 - val_f1_score: 0.3650 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 103s 631ms/step - loss: 1.0587 - f1_score: 0.5316 - val_loss: 0.9920 - val_f1_score: 0.3663 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 104s 636ms/step - loss: 0.9909 - f1_score: 0.5651 - val_loss: 0.7994 - val_f1_score: 0.3797 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 102s 624ms/step - loss: 0.8862 - f1_score: 0.6068 - val_loss: 0.6222 - val_f1_score: 0.3774 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 104s 636ms/step - loss: 0.7769 - f1_score: 0.6526 - val_loss: 0.5891 - val_f1_score: 0.4020 - lr: 5.9514e-05
Epoch 7/20
163/163 [==

## time_delta==45

In [116]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_45'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 106s 629ms/step - loss: 1.0970 - f1_score: 0.3327 - val_loss: 1.0713 - val_f1_score: 0.3680 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 100s 613ms/step - loss: 1.0806 - f1_score: 0.4576 - val_loss: 1.0470 - val_f1_score: 0.3627 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 100s 612ms/step - loss: 1.0517 - f1_score: 0.5389 - val_loss: 0.9443 - val_f1_score: 0.3747 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 100s 616ms/step - loss: 0.9672 - f1_score: 0.5355 - val_loss: 0.8336 - val_f1_score: 0.3723 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 101s 619ms/step - loss: 0.8613 - f1_score: 0.5625 - val_loss: 0.6594 - val_f1_score: 0.3894 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 100s 616ms/step - loss: 0.7670 - f1_score: 0.5860 - val_loss: 0.6307 - val_f1_score: 0.3844 - lr: 5.9514e-05
Epoch 7/20
163/163 [==

## time_delta==60

In [117]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_60'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 110s 649ms/step - loss: 1.0964 - f1_score: 0.3343 - val_loss: 1.0724 - val_f1_score: 0.3662 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 105s 645ms/step - loss: 1.0813 - f1_score: 0.4558 - val_loss: 1.0498 - val_f1_score: 0.3826 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 104s 636ms/step - loss: 1.0576 - f1_score: 0.5477 - val_loss: 0.9919 - val_f1_score: 0.4007 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 104s 638ms/step - loss: 0.9837 - f1_score: 0.5581 - val_loss: 0.6987 - val_f1_score: 0.3942 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 102s 629ms/step - loss: 0.8576 - f1_score: 0.6218 - val_loss: 0.6519 - val_f1_score: 0.3969 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 104s 641ms/step - loss: 0.7573 - f1_score: 0.6495 - val_loss: 0.5635 - val_f1_score: 0.4084 - lr: 5.9514e-05
Epoch 7/20
163/163 [==

## time_delta==75

In [118]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_75'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 88s 502ms/step - loss: 1.0978 - f1_score: 0.3046 - val_loss: 1.0782 - val_f1_score: 0.3711 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 84s 514ms/step - loss: 1.0842 - f1_score: 0.4525 - val_loss: 1.0632 - val_f1_score: 0.3852 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 87s 533ms/step - loss: 1.0628 - f1_score: 0.5508 - val_loss: 1.0248 - val_f1_score: 0.3923 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 86s 527ms/step - loss: 1.0019 - f1_score: 0.5774 - val_loss: 0.7544 - val_f1_score: 0.4071 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 88s 540ms/step - loss: 0.8519 - f1_score: 0.6310 - val_loss: 0.5944 - val_f1_score: 0.3982 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 85s 520ms/step - loss: 0.6754 - f1_score: 0.6656 - val_loss: 0.6272 - val_f1_score: 0.3925 - lr: 5.9514e-05
Epoch 7/20
163/163 [========

## time_delta==90

In [119]:
model_my_lstm_fit_predict(df_comp['price_diff_cat_90'], df_comp['text_clear'])

Epoch 1/20
163/163 [==============================] - 97s 574ms/step - loss: 1.0963 - f1_score: 0.3097 - val_loss: 1.0771 - val_f1_score: 0.3414 - lr: 6.0000e-05
Epoch 2/20
163/163 [==============================] - 97s 597ms/step - loss: 1.0812 - f1_score: 0.4482 - val_loss: 1.0574 - val_f1_score: 0.3593 - lr: 5.9903e-05
Epoch 3/20
163/163 [==============================] - 101s 623ms/step - loss: 1.0565 - f1_score: 0.5469 - val_loss: 1.0124 - val_f1_score: 0.3763 - lr: 5.9805e-05
Epoch 4/20
163/163 [==============================] - 101s 621ms/step - loss: 0.9801 - f1_score: 0.5922 - val_loss: 0.7362 - val_f1_score: 0.3717 - lr: 5.9708e-05
Epoch 5/20
163/163 [==============================] - 102s 625ms/step - loss: 0.8261 - f1_score: 0.6052 - val_loss: 0.6816 - val_f1_score: 0.3809 - lr: 5.9611e-05
Epoch 6/20
163/163 [==============================] - 104s 639ms/step - loss: 0.6556 - f1_score: 0.6371 - val_loss: 0.6500 - val_f1_score: 0.3619 - lr: 5.9514e-05
Epoch 7/20
163/163 [====

# Result

> **Conclusion:** As we can see, there is no much difference between various time deltas taken, that is why we will stop on using **time_delta==30**, as it was adviced in the paper dedicated to this question